# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import os
import numpy as np
import math
import tensorflow as tf

from time import time
from datetime import datetime
from keras.callbacks import TensorBoard

from data_handler_MNIST import MNISTDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions
from CMAPSAuxFunctions import TrainValTensorBoard

from numpy.random import seed
from tensorflow import set_random_seed


/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


# Define different types of Keras models

In [2]:
l2_lambda_regularization = 0.2
l1_lambda_regularization = 0.10
def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(250, input_shape=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dense(10, activation='softmax',kernel_initializer='glorot_normal', name='out'))
    
    return model

# Create Tunable Model

In [3]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0,beta_1=0.5)
    
    lossFunction = "categorical_crossentropy"
    metrics = ["accuracy"]
    model = None

    #Create and compile the models
    model = RULmodel_SN_5(shape)
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    return model




In [4]:
# num_features = len(selected_features)

shape = (784,)
dHandler_mnist = MNISTDataHandler()
model = get_compiled_model(shape, model_type='ann')
tModel = SequenceTunableModelRegression('ModelMNIST_SN_1', model, lib_type='keras', data_handler=dHandler_mnist)


# Load Data

In [5]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.load_data(verbose=1, cross_validation_ratio=0.2)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data. Cros-Validation ratio 0.2


/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Printing shapes

Training data (X, y)
(48000, 784)
(48000, 10)
Cross-Validation data (X, y)
(12000, 784)
(12000, 10)
Testing data (X, y)
(10000, 784)
(10000, 10)
Printing first 5 elements

Training data (X, y)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
Cross-Validation data (X, y)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
Testing data (X, y)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.

# Train the model and test some Tunable Model functionalities

In [6]:
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

shape = (784,)


model = get_compiled_model(shape, model_type='ann')
tModel.change_model('ModelRUL_SN_1', model, 'keras')

# now = datetime.now()
# tensorboard = TrainValTensorBoard(log_dir="./logs/"+now.strftime("%Y%m%d-%H%M%S")+ "/",write_graph=False)

tModel.epochs = 20
tModel.train_model(learningRate_scheduler=lrate, verbose=1)




training with cv
Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 2s 35us/step - loss: 0.5476 - acc: 0.8622 - val_loss: 0.2879 - val_acc: 0.9192
Epoch 2/20
48000/48000 [==============================] - 1s 30us/step - loss: 0.2380 - acc: 0.9345 - val_loss: 0.2237 - val_acc: 0.9367
Epoch 3/20
48000/48000 [==============================] - 1s 28us/step - loss: 0.1820 - acc: 0.9493 - val_loss: 0.1865 - val_acc: 0.9469
Epoch 4/20
48000/48000 [==============================] - 2s 32us/step - loss: 0.1460 - acc: 0.9593 - val_loss: 0.1622 - val_acc: 0.9531
Epoch 5/20
48000/48000 [==============================] - 1s 29us/step - loss: 0.1222 - acc: 0.9657 - val_loss: 0.1464 - val_acc: 0.9565
Epoch 6/20
48000/48000 [==============================] - 1s 30us/step - loss: 0.1044 - acc: 0.9714 - val_loss: 0.1315 - val_acc: 0.9613
Epoch 7/20
48000/48000 [==============================] - 2s 32us/step - loss: 0.0907 - acc: 0.9750 - val_loss:

In [7]:
tModel.evaluate_model(cross_validation=True)

cScores = tModel.scores
print(cScores)

12000/12000 [==============================] - 1s 56us/step
{'loss': 0.08689974501760056, 'score_1': 0.97425}


In [8]:
tModel.evaluate_model(cross_validation=False)

cScores = tModel.scores
print(cScores)

10000/10000 [==============================] - 1s 53us/step
scores
{'loss': 0.07322948413498234, 'score_1': 0.978}
